# Maps of trend per grid cell

In [18]:
import sys; sys.path.append('/rds/general/user/cb2714/home/wwa'); from wwa import *
warnings.filterwarnings("ignore", category = FutureWarning) # otherwise there will be a LOT of warnings

def merge_byindex(dfs): return pd.concat(dfs, axis=0)

## Reshape dataframe into maps

In [19]:
def vec2map(x, mask):
    
    # reconstruct vector into map
    # create an empty map with NA in same cells as masks
    arr = mask.where(np.isnan(mask), 0)
    
    # get coordinates of non-empty cells
    px = np.argwhere(~np.isnan(mask.values))
    
    # Transfer vector values into non-empty cells in array
    if len(px) == len(x):
        for i in list(range(len(px))): arr[px[i,0], px[i,1]] = x[i]
        return arr
    else:
        print(str(len(x))+" values, but "+str(len(px))+" cells")
        return

In [20]:
# load target map showing original shape of data
tmplt = xr.open_dataset("map-tmplt_tx5x_era5.nc").tmax

In [21]:
# load all the flattened results and compile into a single dataframe
df = merge_byindex([pd.read_csv(fnm, index_col = 0) for fnm in sorted(glob.glob("flattened-res/*.csv"))])

In [32]:
# reconstruct map
res = xr.merge([vec2map(df.loc[r], tmplt).rename(r) for r in df.index])

# replace infinite values with very high value
res["PR"] = res.PR.where(np.isfinite(res.PR), 100).where(~np.isnan(res.PR))

In [33]:
# save
res.to_netcdf("gridded-trend_tx5x_era5.nc")

## Plot maps

In [36]:
plot_kwargs = {"event_magnitude" : {"cmap" : "YlOrRd", "vmin" : 30, "vmax" : 45},
               "return_period"   : {"cmap" : "viridis", "norm" : matplotlib.colors.LogNorm(vmin = 1, vmax = 1000)},
               "PR"     : {"cmap" : "RdPu", "vmin" : 0, "vmax": 10},
               "dI_abs" : {"cmap" : "RdBu_r", "vmin" : -2, "vmax" : 2},
              }

main = {
        "alpha_gmst" : "Change in Tx5X (°C) per 1C increase in GMST",
        "dI_abs" : "Change in Tx5X\ndue to 1.3C increase in GMST",
        "event_magnitude" : "Hottest 5-day period\nduring May-June 2024",
        "PR" : "Change in likelihood\ndue to 1.3C increase in GMST",
        "return_period" : "Return period of\nhottest 5-day period",
       }

cbar_details = {
         "alpha_gmst" : {"label" : "Change in temperature (°C)", "extend" : "both"},
         "dI_abs" : {"label" : "Change in temperature (°C)", "extend" : "both"},
         "event_magnitude" : {"label" : "Temperature (°C)", "extend" : "both"},
         "PR" : {"label" : "Probability ratio", "extend" : "max"},
         "return_period" : {"label" : "Return period (years)", "extend" : "max", "format" : lambda x, pos: str(int(x))},
        }

In [ ]:
fig, axs = plt.subplots(ncols = 4, figsize = (16,5), dpi = 100, subplot_kw={"projection" : cartopy.crs.PlateCarree()})
plt.subplots_adjust(wspace = 0.05)

cbar_kwargs = {"location" : "bottom", "fraction" : 0.065, "shrink" : 0.9, "pad" : 0.1, "extend" : "both"}

plot_vars = ["event_magnitude", "return_period", "dI_abs", "PR"]

for i in range(4):
    
    ax = axs[i]
    pv = plot_vars[i]
    
    res[pv].plot(ax = ax, **plot_kwargs[pv], cbar_kwargs = cbar_kwargs | cbar_details[pv])
    
    ax.coastlines()
    ax.add_feature(cartopy.feature.BORDERS, color = "k", ls = "--", alpha = 0.5)
    
    ax.set_title(main[pv])
    
    ax.gridlines(color = "k", alpha = 0.1, xlocs = range(-120,-80,5), draw_labels = ["bottom"])
    
axs[0].gridlines(alpha = 0, draw_labels = ["left"])
